<a href="https://colab.research.google.com/github/VTU20379/VelTech-GenAI/blob/main/Bert_mlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from scipy.special import softmax
import numpy as np

In [ ]:
model_name = "bert-base-cased"

# Instantiate the tokenizer and model for the specified pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)
print(tokenizer)
print(model)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_id

In [ ]:
# Get the mask token from the tokenizer
mask = tokenizer.mask_token
print(mask)

[MASK]


In [ ]:
# Create a sentence with a mask token to be filled in by the model
sentence = f"It is good to have {mask} for lunch."
# Tokenize the sentence
tokens = tokenizer.tokenize(sentence)
print(tokens)

['It', 'is', 'good', 'to', 'have', '[MASK]', 'for', 'lunch', '.']


In [ ]:
# Create a sentence with a mask token to be filled in by the model
sentence = f"I want to meet {mask} tonight."
# Tokenize the sentence
tokens = tokenizer.tokenize(sentence)
print(tokens)

['I', 'want', 'to', 'meet', '[MASK]', 'tonight', '.']


In [ ]:
# Encode the sentence using the tokenizer and return the input tensors
encoded_inputs = tokenizer(sentence, return_tensors='pt')
print(encoded_inputs)

{'input_ids': tensor([[ 101,  146, 1328, 1106, 2283,  103, 3568,  119,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
#encoded_inputs=tokenizer(sentence)
#print(encoded_inputs)

{'input_ids': [101, 146, 1328, 1106, 2283, 103, 3568, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
outputs = model(**encoded_inputs)
outputs


MaskedLMOutput(loss=None, logits=tensor([[[ -7.2080,  -7.1081,  -7.2716,  ...,  -6.0417,  -5.8467,  -6.1377],
         [ -7.9152,  -8.2049,  -7.9754,  ...,  -6.6298,  -6.4941,  -6.8368],
         [-12.4463, -12.3040, -12.1644,  ...,  -6.9075,  -7.5696,  -7.8245],
         ...,
         [ -7.7738,  -8.0661,  -7.0037,  ...,  -5.7493,  -5.9397,  -7.0409],
         [-11.7280, -11.7840, -11.6647,  ...,  -8.4709,  -9.0766,  -9.8484],
         [-13.4606, -14.2306, -13.9173,  ...,  -9.6105, -11.8186, -10.8183]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [ ]:
logits=outputs.logits.detach().numpy()[0]
logits.shape

(9, 28996)

In [ ]:
len(tokens)

7

In [ ]:
mask_logits=logits[tokens.index(mask)+1]
mask_logits

array([-5.632214 , -5.705392 , -5.5025053, ..., -5.3011913, -5.295761 ,
       -5.3564987], dtype=float32)

In [ ]:
confidence=softmax(mask_logits)
confidence

array([1.0081819e-08, 9.3704058e-09, 1.1478123e-08, ..., 1.4037833e-08,
       1.4114296e-08, 1.3282541e-08], dtype=float32)

In [ ]:
confidence.sum()

0.9999999

In [ ]:
for i in np.argsort(confidence_scores)[::-1][:5]:
    pred_token = tokenizer.decode(i)
    score = confidence_scores[i]

    # Print the predicted sentence with the mask token replaced by the predicted token, and the confidence score
    print(sentence.replace(mask, pred_token), score)

In [ ]:
for i in np.argsort(confidence)[::-1][:5]:
  pred=tokenizer.decode(i)
  score=confidence[i]
  print(sentence.replace(mask,pred),score)

I want to meet you tonight. 0.3609081
I want to meet him tonight. 0.18192914
I want to meet her tonight. 0.14262769
I want to meet someone tonight. 0.03402644
I want to meet again tonight. 0.022109564


In [ ]:
for i in np.argsort(confidence)[::-1][:5]:
  pred=tokenizer.decode(i)
  score=confidence[i]
  print(sentence.replace(mask,pred),score)

I want to meet you tonight. 0.3609081
I want to meet him tonight. 0.18192914
I want to meet her tonight. 0.14262769
I want to meet someone tonight. 0.03402644
I want to meet again tonight. 0.022109564
